In [16]:
import pandas as pd

In [17]:
def get_player_attribute_data(year):
    import sqlite3 as db 
    conn = db.connect('soccer/database.sqlite')
    query_str = """select  p.player_name,pa2.* from player_attributes pa2, player p where p.player_api_id = pa2.player_api_id and
    (pa2.player_api_id, pa2.date) in (
    select z.player_api_id, z.date from (
    select pa.player_api_id, pa.date, min(abs(strftime('%%s',pa.date) - strftime('%%s','%(year)s-09-01 00:00:00'))) from player_attributes pa, player p 
                                    where pa.player_api_id = p.player_api_id
                                    and strftime('%%s',pa.date) > strftime('%%s','2014-05-14 00:00:00') 
                                    and pa.date like '%(year)s%%'
                                    group by pa.player_api_id ) z )"""%{'year':year}
    
    all_player_attrs = pd.read_sql_query(query_str,conn)
    return all_player_attrs


In [84]:

def get_player_attr_pos(year):
    """JOIN player attribute data with player position data based on PLAYER_NAME """
    player_position_data = pd.read_csv('player_pos_year.csv', dtype = {'overall_rating':float} )
    player_position_data = player_position_data[player_position_data['Year'] == (int(year) + 1)]
    
    player_attribute_data  = get_player_attribute_data(year)
        
    joined = player_attribute_data.merge(player_position_data,on = "player_name" , how='inner')
    return joined

In [85]:

def create_best_team(full_team):
    pos_index = {0:[0],1:[1,2],2:[3],3:[4,6], 4:[5],5:[7],6:[8],7:[9],8:[10]}

    field_pos = {0:[['LB', 'LWB'],1], 1:[['CB'],2], 2:[['RB', 'RWB'],1], 3:[['CM'],2], 4: [['CAM', 'CDM'],1],5: [['LM', 'LW'],1], 6: [['CF', 'ST'],1], 7:[['RM', 'RW'],1], 8:[['GK'],1]}    

    alt_pos = {3: ['CAM','CDM', 'LM', 'RM'], 4 : ['CM'], 5: ['RM','RW','CF','ST'], 6 : ['RM','LM'], 7 : ['LM','LW','ST','CF']}

    iter_count = 0
    df_start = pd.DataFrame()
    #full_team = data[data['Team'].str.startswith(name)]
    full_team_size = len(full_team)
    temp_team = full_team.sort_values(by = ['player_name','overall_rating']).drop_duplicates(subset = ['player_name'], keep = 'first').copy()
    
    find_alternate = False
    
    while field_pos:
        if iter_count == full_team_size:
            return None
        for key in list(field_pos.keys()):

            (positions, count) = field_pos[key]
            
            if find_alternate:
                positions = alt_pos[key]
                pos_players = temp_team[temp_team['Position'].isin(positions)].sort_values(by = ['overall_rating'], ascending = False).head(count)
            else:
                pos_players = temp_team[temp_team['Position'].isin(positions)].sort_values(by = ['overall_rating'], ascending = False).head(count)
                
            if len(pos_players) > 0:
                
                shirt_numbers = pos_index[key][:len(pos_players)]
                
                pos_players['pos_index'] = shirt_numbers
                pos_index[key] = pos_index[key][len(pos_players):]
                df_start = df_start.append(pos_players)
                temp_team = temp_team[~temp_team['player_name'].isin(pos_players['player_name'])]

                if len(pos_players)!=count:
                    missing_count = count - len(pos_players)
                    field_pos[key][1] = missing_count
                else:
                    del field_pos[key]
        find_alternate = True
        iter_count +=1
        
    return df_start

In [103]:
data = get_player_attr_pos('2015')
# all_teams = data['Team'].unique()

# for team in all_teams:
#     full = data[data['Team'] == team]
#     start = create_best_team(full)
#     print(start[['player_name','Position','overall_rating','pos_index']])
#     if start is None:
#         print("ERROR==========")


west_ham = data[data['Team'].str.startswith('West Ham United')]

starting = create_best_team(west_ham)
print(starting[['overall_rating','pos_index']].set_index(['pos_index']).transpose())

pos_index         0     1     2     3     4     5     7     8     9     10  \
overall_rating  73.0  78.0  73.0  74.0  76.0  76.0  81.0  78.0  81.0  78.0   

pos_index         6   
overall_rating  80.0  


In [ ]:
def get_all_team_data(year):
    
    data = {}

    joined_data = get_full_data(year)

    f = get_fixtures(year)
    teams = f['HomeTeam'].unique() #get all unique teams

    for team in teams:
        team_data = create_team(joined_data, team)
        team_attr = agg_attrs_all(team_data)
        data[team] = (team_data, team_attr)
    return data